In [1]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D
from pathlib import Path
import numpy as np
import pandas as pd
from random import seed
from random import randint
from sklearn.utils import shuffle
import os
import client_util

Using TensorFlow backend.


In [2]:
cwd = os.getcwd()
client_util.change_dir("avg_weights",cwd)

In [3]:
def test_model(x,y_actual,keep_prob):
    #first conv layer
    W1 = np.load("conv1_W1_avg.npy",allow_pickle=True)
    b1 = np.load("conv1_b1_avg.npy",allow_pickle=True)
    conv_1 = tf.nn.relu(tf.nn.conv2d(x, W1, [1, 1, 1, 1], padding='SAME') + b1)
    pool_1 = tf.nn.max_pool(conv_1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool_1')
    
    #second conv layer
    W2 = np.load("conv2_W2_avg.npy",allow_pickle=True)
    b2 = np.load("conv2_b2_avg.npy",allow_pickle=True)
    conv_2 = tf.nn.relu(tf.nn.conv2d(pool_1, W2, [1, 1, 1, 1], padding='SAME') + b2)
    pool_2 = tf.nn.max_pool(conv_2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool_2')

    #flatten pool_2
    dim2 = pool_2.get_shape()[1].value * pool_2.get_shape()[2].value * pool_2.get_shape()[3].value
    pool_2_flat = tf.reshape(pool_2, [-1, dim2])
    
    #first fully connected layer
    W_fc1 = np.load("fc1_W3_avg.npy",allow_pickle=True)
    b_fc1 = np.load("fc1_b3_avg.npy",allow_pickle=True)
    fc_1 = tf.nn.relu(tf.matmul(pool_2_flat, W_fc1) + b_fc1)

    #add dropout layer
    fc_1_drop = tf.nn.dropout(fc_1, keep_prob)
    
    #output layer
    W_fc2 = np.load("w4_avg.npy",allow_pickle=True)
    b_fc2 = np.load("b4_avg.npy",allow_pickle=True)
    y_pred = tf.matmul(fc_1_drop, W_fc2) + b_fc2

    print("===========test model loaded==============")
    return y_pred

In [4]:
def main():
    datasets = ['cifar10','mnist','fashion_mnist']
    curr_testingData = datasets[0]
    #load dataset
    x_train,y_train,x_test,y_test = client_util.load_dataset(curr_testingData)
    print("x_train shape: {}, y_train shape: {}".format(x_train.shape,y_train.shape))
    print("x_test shape: {}, y_test shape: {}".format(x_test.shape,y_test.shape))
    
    #expand dimensions if mnist or fashion_mnist since cnn expect 4 dimension
    if curr_testingData != "cifar10":
        x_train = np.expand_dims(x_train,axis = -1)
        x_test = np.expand_dims(x_test,axis = -1)
        
    #normalize data
    x_train,y_train,x_test,y_test = client_util.normalize_data(x_train,y_train,x_test,y_test)
    
    #training parameters
    if curr_testingData == "cifar10":
        img_width = 32
        img_height = 32
        num_channels = 3
    else:
        img_width = 28
        img_height = 28
        num_channels = 1
    num_classes = 10
    
    #create placeholders
    x = tf.placeholder(tf.float32,shape=[None,img_width,img_height,num_channels])
    y_actual = tf.placeholder(tf.float32,shape=[None,num_classes])
    keep_prob = tf.placeholder(tf.float32)
    
    #get prediction
    y_pred = test_model(x,y_actual,keep_prob)
    
    with tf.variable_scope('acc',reuse = tf.AUTO_REUSE):
        matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_actual,1))
        acc = tf.reduce_mean(tf.cast(matches,tf.float32)) 
    
    with tf.compat.v1.Session() as sess:
        sess.run(tf.global_variables_initializer())
        test_acc_list = []
    
        for epoch in range(10):
            #shuffle data 

            test_acc = acc.eval(feed_dict = {x:x_test, y_actual:y_test,keep_prob: 0.5})
            test_acc_list.append(test_acc)

            if epoch%2==0:
                print("Epoch: {}, Testing accuracy: {}".format(epoch,test_acc))
            
        cwd = os.getcwd()
    
    
if __name__ == '__main__':
    main()

x_train shape: (60000, 28, 28), y_train shape: (60000,)
x_train type: <class 'numpy.ndarray'>, y_train type: <class 'numpy.ndarray'>
x_train shape: (60000, 28, 28), y_train shape: (60000,)
x_test shape: (10000, 28, 28), y_test shape: (10000,)
x_train shape: (60000, 28, 28, 1), y_train shape: (60000, 10)
x_train type: <class 'numpy.ndarray'>, y_train type: <class 'numpy.ndarray'>
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
===========test model loaded==============
Epoch: 0, Testing accuracy: 0.09679999947547913
Epoch: 2, Testing accuracy: 0.09749999642372131
Epoch: 4, Testing accuracy: 0.09560000151395798
Epoch: 6, Testing accuracy: 0.09080000221729279
Epoch: 8, Testing accuracy: 0.10140000283718109
